<a href="https://colab.research.google.com/github/Shreyabaral/Meeting-Assistant/blob/main/MeetingAssistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install all the required libraries
!pip install streamlit langchain  faiss-cpu


In [ ]:
pip install openai-whisper

In [ ]:
import whisper
#function to transcribe audio
def transcribe_audio(audio_file):
    model = whisper.load_model("base")
    result = model.transcribe(audio_file)
    return result["text"]

In [ ]:
!pip install datasets py7zr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 28.0 MB/s eta 0:00:00


In [ ]:
#transcribe_audio('/content/New Recording.m4a')

Fine tuning using PEFT

In [ ]:
from datasets import load_dataset

ds = load_dataset("samsum")

corpus.7z:   0%|          | 0.00/2.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
ds['train'][10]

{'id': '13727633',
 'dialogue': 'Lucas: Hey! How was your day?\r\nDemi: Hey there! \r\nDemi: It was pretty fine, actually, thank you!\r\nDemi: I just got promoted! :D\r\nLucas: Whoa! Great news!\r\nLucas: Congratulations!\r\nLucas: Such a success has to be celebrated.\r\nDemi: I agree! :D\r\nDemi: Tonight at Death & Co.?\r\nLucas: Sure!\r\nLucas: See you there at 10pm?\r\nDemi: Yeah! See you there! :D',
 'summary': 'Demi got promoted. She will celebrate that with Lucas at Death & Co at 10 pm.'}

In [ ]:
ds['test'][0]

{'id': '13862856',
 'dialogue': "Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye",
 'summary': "Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry."}

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

In [ ]:
# Initial reuslt of the model

# Sample input (adjust as needed)
input_text = """Rann: Hey Harry, how have you been? Long time no see!Harry: Hey! What a surprise!Harry: Yes, you are right, we haven’t seen each other in a long time. How have you been?Rann: There is an important campaign next week which is keeping me busy otherwise rest is going good in my life.Rann: How about you?
Harry: Oh! I just finished a meeting with a very important client of mine and now I finally have some free time. I feel relieved that I’m done with it.
Rann: Good for you then. Hey! Let’s make a plan and catch up with each other after next week.Rann: What do you say?Harry: Sure, why not? Give me a call when you are done with your project.Rann: Sure, then.Rann: Bye, take care.Harry: Bye buddy.
:
"""


#Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt", max_length=2024, truncation=True)

# Generate output
output_ids = model.generate(
    inputs["input_ids"],
    max_length=1000,         # Adjust the maximum length as needed
    num_beams=16,            # Number of beams for beam search (can be tuned)
    length_penalty=35.0,     # Length penalty to control output length
    early_stopping=True     # Stop when the end token is generated
)

# Decode the generated tokens to text
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(f"Generated Text without finetuning: {output_text}")



Generated Text: Rann: Hey Harry, how have you been? Long time no see! Harry: Hey! What a surprise! Rann: There is an important campaign next week which is keeping me busy otherwise rest is going good in my life. Harry: Sure, why not? Give me a call when you are done with your project.


In [ ]:
tokenized = tokenizer(""" New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in p" """)
print(type(tokenized))
print(tokenized)

<class 'transformers.tokenization_utils_base.BatchEncoding'>
{'input_ids': [0, 188, 469, 36, 16256, 43, 1779, 226, 8878, 1731, 39374, 366, 21, 883, 107, 793, 6, 79, 300, 2997, 11, 580, 17419, 413, 6, 188, 469, 4, 50118, 250, 76, 423, 6, 79, 300, 2997, 456, 11, 580, 17419, 413, 6, 53, 7, 10, 430, 313, 8, 396, 35447, 11162, 69, 78, 1623, 4, 50118, 19933, 504, 360, 71, 14, 3397, 6, 79, 300, 478, 3804, 648, 456, 4, 1892, 6, 1731, 39374, 366, 2998, 22, 100, 109, 113, 292, 55, 498, 6, 2128, 129, 624, 80, 688, 9, 349, 97, 4, 50118, 1121, 1824, 6, 79, 2997, 683, 55, 6, 42, 86, 11, 5, 18170, 4, 96, 41, 2502, 13, 10, 3397, 4385, 6, 79, 2305, 24, 21, 69, 22, 9502, 8, 129, 113, 3397, 4, 50118, 14507, 39374, 366, 6, 122, 3191, 6, 16, 2114, 80, 1837, 3948, 9, 22, 1529, 2961, 10, 3950, 10320, 13, 3386, 11, 5, 78, 3093, 60, 5056, 7, 69, 3950, 1997, 15, 5, 50118, 24789, 3397, 4385, 2502, 6, 309, 7, 461, 2339, 4, 50118, 44693, 26, 5, 21304, 58, 233, 9, 41, 2447, 10656, 4, 50118, 4148, 273, 6, 79, 4407, 

In [ ]:
print("Input IDs:", tokenized["input_ids"])
print("Attention Mask:", tokenized["attention_mask"])

Input IDs: [0, 188, 469, 36, 16256, 43, 1779, 226, 8878, 1731, 39374, 366, 21, 883, 107, 793, 6, 79, 300, 2997, 11, 580, 17419, 413, 6, 188, 469, 4, 50118, 250, 76, 423, 6, 79, 300, 2997, 456, 11, 580, 17419, 413, 6, 53, 7, 10, 430, 313, 8, 396, 35447, 11162, 69, 78, 1623, 4, 50118, 19933, 504, 360, 71, 14, 3397, 6, 79, 300, 478, 3804, 648, 456, 4, 1892, 6, 1731, 39374, 366, 2998, 22, 100, 109, 113, 292, 55, 498, 6, 2128, 129, 624, 80, 688, 9, 349, 97, 4, 50118, 1121, 1824, 6, 79, 2997, 683, 55, 6, 42, 86, 11, 5, 18170, 4, 96, 41, 2502, 13, 10, 3397, 4385, 6, 79, 2305, 24, 21, 69, 22, 9502, 8, 129, 113, 3397, 4, 50118, 14507, 39374, 366, 6, 122, 3191, 6, 16, 2114, 80, 1837, 3948, 9, 22, 1529, 2961, 10, 3950, 10320, 13, 3386, 11, 5, 78, 3093, 60, 5056, 7, 69, 3950, 1997, 15, 5, 50118, 24789, 3397, 4385, 2502, 6, 309, 7, 461, 2339, 4, 50118, 44693, 26, 5, 21304, 58, 233, 9, 41, 2447, 10656, 4, 50118, 4148, 273, 6, 79, 4407, 45, 2181, 23, 331, 2124, 837, 11, 5, 18170, 6, 309, 7, 69, 1921,

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

# LoRA configuration for efficient fine-tuning
lora_config = LoraConfig(
    r=8,  # Rank (low-rank approximation) value
    lora_alpha=16,  # Scaling factor
    lora_dropout=0.1,  # Dropout for regularization
    task_type=TaskType.SEQ_2_SEQ_LM,  # Summarization task (Seq2Seq)
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)


In [ ]:
def preprocess_function(examples):
    inputs = examples["dialogue"]
    targets = examples["summary"]

    # Tokenize inputs and targets for model
    model_inputs = tokenizer(inputs, max_length=787, truncation=True)
    labels = tokenizer(targets, max_length=150, truncation=True)

    # Add labels to inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the preprocessing to the dataset
dataset = ds.map(preprocess_function, batched=True)


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
import os

os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq

# Training arguments
'''
training_args = TrainingArguments(
    output_dir="./results",          # Output directory
    eval_strategy="epoch",     # Evaluate every epoch
    logging_dir="./logs",            # Logging directory
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    num_train_epochs=3,              # Number of training epochs
    logging_steps=10,                # Steps between logs
    save_steps=10,                   # Steps between checkpoints
)


'''

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    evaluation_strategy="steps",
    logging_dir="./logs",
    per_device_train_batch_size=4,  # Larger batch size
    per_device_eval_batch_size=4,
    num_train_epochs=1,             # More epochs
    logging_steps=10,
    save_steps=10,
    learning_rate=3e-5,             # Adjusted learning rate
    warmup_steps=500,
    weight_decay=0.01,
    max_grad_norm=1.0,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss,Validation Loss


In [ ]:
trainer.save_model("./final_model")  # Saves the model
tokenizer.save_pretrained("./final_model")

('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/vocab.json',
 './final_model/merges.txt',
 './final_model/added_tokens.json')

In [ ]:
# Evaluate on test set
results = trainer.evaluate()

# Print results
print(results)


{'eval_loss': 1.7835663557052612, 'eval_runtime': 30.7832, 'eval_samples_per_second': 27.97, 'eval_steps_per_second': 7.017, 'epoch': 3.0}


In [ ]:
# Load the fine-tuned model and tokenizer
model_name = "./final_model"  # Path to the saved model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)



In [ ]:
# Sample input (adjust as needed)
input_text = """
Rann: Hey Harry, how have you been? Long time no see!Harry: Hey! What a surprise!Harry: Yes, you are right, we haven’t seen each other in a long time. How have you been?
Rann: There is an important campaign next week which is keeping me busy otherwise rest is going good in my life.Rann: How about you?
Harry: Oh! I just finished a meeting with a very important client of mine and now I finally have some free time. I feel relieved that I’m done with it.
Rann: Good for you then. Hey! Let’s make a plan and catch up with each other after next week.Rann: What do you say?Harry: Sure, why not? Give me a call when you are done with your project.Rann: Sure, then.Rann: Bye, take care.Harry: Bye buddy.
:"""



In [ ]:
# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt", max_length=2024, truncation=False)

# Generate output
output_ids = model.generate(
    inputs["input_ids"],
    max_length=1000,         # Adjust the maximum length as needed
    num_beams=8,            # Number of beams for beam search (can be tuned)
    length_penalty=35.0,     # Length penalty to control output length
    early_stopping=True     # Stop when the end token is generated
)

# Decode the generated tokens to text
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(f"Generated Text: {output_text}")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')